<a href="https://colab.research.google.com/github/sonjoy1s/ML/blob/main/M_odule18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

data = load_breast_cancer()

In [11]:
X = pd.DataFrame(data.data,columns=data.feature_names)
y = pd.Series(data.target)
print(X.shape)
print(y.shape)

(569, 30)
(569,)


In [13]:
y.value_counts()

,count
1,357
0,212
